In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral-7b-instruct/gguf/mistral-7b-instructv0.1/1/mistral-7b-instruct-v0.1.Q8_0.gguf


In [6]:
#!pip install langchain llama-cpp-python 

In [7]:
#!pip install sentence-transformers

In [8]:
#!pip install chromadb

# Import LLM model

In [9]:
# import packages from langchains for llm

from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
model_path = '/kaggle/input/mistral-7b-instruct/gguf/mistral-7b-instructv0.1/1/mistral-7b-instruct-v0.1.Q8_0.gguf'

In [11]:
callbackManager = CallbackManager([StreamingStdOutCallbackHandler()])

In [12]:
n_gpu_layers = 2  # Metal set to 1 is enough.
n_batch = 512  

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=False,
)

# Scrap Data from Website

In [13]:
# import python packages to scrap data from sites

from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader

In [113]:
web_path = 'https://www.geeksforgeeks.org/introduction-to-langchain/'

In [114]:
# now load the document

loader = WebBaseLoader(
    web_path = web_path,
    header_template = None,
    verify_ssl = True,
    proxies = None,
    continue_on_failure = False,
    requests_per_second = 2,
    default_parser = 'html.parser'
)

In [115]:
docs = loader.load()
txt_data = docs[0].page_content

In [117]:
len(txt_data)

22311

In [118]:
txt_data[0:500]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIntroduction to LangChain - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\nTutorialsPython TutorialTaking Input in PythonPython OperatorsPython Data TypesPython NumbersPython StringPython ListsPython TuplesSets in PythonPython DictionaryPython Loops and Control FlowPython If ElsePython For LoopsPython While LoopsPython BreaksPython Continue StatementPython Pass StatementPython FunctionsPython OOPS ConceptPython Dat'

# Text Splitter

In [119]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

In [120]:
# now define the chunks and overlap

chunks_size = 500
chunks_overlap = 40

rec_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    chunk_size=chunks_size,
    chunk_overlap=chunks_overlap,
    is_separator_regex=False
)

In [121]:
rec_splitter.split_text(txt_data[0:1000])

['Introduction to LangChain - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content',
 'TutorialsPython TutorialTaking Input in PythonPython OperatorsPython Data TypesPython NumbersPython StringPython ListsPython TuplesSets in PythonPython DictionaryPython Loops and Control FlowPython If ElsePython For LoopsPython While LoopsPython BreaksPython Continue StatementPython Pass StatementPython FunctionsPython OOPS ConceptPython Data StructuresPython DSALinked ListStackQueueTreeHeapHashingGraphSetsMapAdvance Data StructureSorting AlgorithmsSearching AlgorithmsPython Exception',
 'AlgorithmsPython Exception HandlingPython File HandlingPython ExercisesPython List ExercisePython String ExercisePython Tuple ExercisePython Dictionary ExercisePython Set ExercisePython Design PatternsPython Programming ExamplesPython Practice QuestionsJavaJava Programming LanguageJava TutorialData TypesVariablesOperators

In [122]:
# now try the Simple charater splitter

splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=chunks_size,
    chunk_overlap = chunks_overlap
)

In [123]:
splitter.split_text(txt_data[0: 1500])

['Introduction to LangChain - GeeksforGeeks\nSkip to content',
 'TutorialsPython TutorialTaking Input in PythonPython OperatorsPython Data TypesPython NumbersPython StringPython ListsPython TuplesSets in PythonPython DictionaryPython Loops and Control FlowPython If ElsePython For LoopsPython While LoopsPython BreaksPython Continue StatementPython Pass StatementPython FunctionsPython OOPS ConceptPython Data StructuresPython DSALinked ListStackQueueTreeHeapHashingGraphSetsMapAdvance Data StructureSorting AlgorithmsSearching AlgorithmsPython Exception HandlingPython File HandlingPython ExercisesPython List ExercisePython String ExercisePython Tuple ExercisePython Dictionary ExercisePython Set ExercisePython Design PatternsPython Programming ExamplesPython Practice QuestionsJavaJava Programming LanguageJava TutorialData TypesVariablesOperatorsOperators in JavaArithmetic OperatorsUnary OperatorsAssignment OperatorsRelational OperatorsLogical OperatorsTernary OperatorsFlow Control in JavaFlo

In [124]:
split = rec_splitter.split_documents(
    docs
)

In [125]:
embedding = HuggingFaceEmbeddings(
    model_name='all-MiniLM-L6-v2'
)

In [126]:
t1= "I love dogs"
t2 = "I love cats"
t3 = "Study is very hard"

In [127]:
embd1 = embedding.embed_query(t1)
embd2 = embedding.embed_query(t2)
embd3 = embedding.embed_query(t3)

In [128]:
np.dot(embd1,embd2), np.dot(embd2,embd3)

(0.7720984757242813, 0.06933481201584218)

# Add Vector Databases

In [129]:
from langchain.vectorstores import Chroma
from langchain.vectorstores import pinecone

In [130]:
db = Chroma.from_documents(
    documents= split,
    embedding=embedding,
    collection_name='langchain',
    persist_directory='docs1/chroma/'
)

In [135]:
query = 'why we need langchain?'

In [136]:
db.similarity_search(
    query=query,
    k=3
)

[Document(page_content='LangChain Key Concepts:\nThe main properties of LangChain Framework are :', metadata={'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US', 'source': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'title': 'Introduction to LangChain - GeeksforGeeks'}),
 Document(page_content='Applications of LangChain\nLangChain is a powerful tool that can be used to build a wide range of LLM-powered applications. It is simple to use and has a large user and contributor community.', metadata={'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US', 'source': 'https://www.geeksforgee

In [137]:
# using MMR

mmr_docs = db.max_marginal_relevance_search(
    query=query,
    k=2,
    fetch_k=2
)

In [138]:
mmr_docs

[Document(page_content='LangChain Key Concepts:\nThe main properties of LangChain Framework are :', metadata={'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US', 'source': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'title': 'Introduction to LangChain - GeeksforGeeks'}),
 Document(page_content='Applications of LangChain\nLangChain is a powerful tool that can be used to build a wide range of LLM-powered applications. It is simple to use and has a large user and contributor community.', metadata={'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US', 'source': 'https://www.geeksforgee

# Compressor Technique

In [139]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [140]:
# meta_data_field_info = [
#     AttributeInfo(
#         name='source',
#         description='Text chunks should be one of `/docs/use_cases/query_analysis/`',
#         type='string'
#     ),
#     AttributeInfo(
#         name='source',
#         description='The heading from contexts',
#         type='string'
#     )
# ]

In [141]:
# docs_content_disc = 'Langchain RAG'

# ret = SelfQueryRetriever.from_llm(
#     llm,
#     db,
#     docs_content_disc,
#     meta_data_field_info,
#     verbose=True
# )

In [142]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [143]:
compressor = LLMChainExtractor.from_llm(
    llm=llm
)

In [144]:
compressor_ret = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=db.as_retriever(
        search_type = 'mmr'
    )
)

In [146]:
question = "What is the langchain framework?"

In [147]:
compressed_docs = compressor_ret.get_relevant_documents(
    query=question
)

/opt/conda/lib/python3.10/site-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [148]:
pretty_print_docs(compressed_docs)

Document 1:

None
----------------------------------------------------------------------------------------------------
Document 2:

langchain framework, prompt templates, chains, agents, usage, AI-powered applications, personal assistants, prompt management, automating tasks.
----------------------------------------------------------------------------------------------------
Document 3:

LangChain, LLM, GPT-4, Python, JavaScript
>>>
----------------------------------------------------------------------------------------------------
Document 4:

Introduction to LangChain - GeeksforGeeks


# LLM + RAG

In [149]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [150]:
# Create prompt template for LLM

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""

In [151]:
qa_chain_prompt = PromptTemplate.from_template(
    template=template
)

In [152]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = db.as_retriever(
        search_type='mmr'
    ),
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": qa_chain_prompt
    }
)

In [153]:
question = "Why we need langchain?"

In [154]:
result = qa_chain({"query":question})

In [155]:
result["result"]

' LangChain is a powerful tool for data augmentation and text classification with various LLMs that can be used with it. It is easy to set up and use with various libraries like OpenAI, making it an essential tool for creating new datasets and training machine learning models. Thanks for asking!'